In [102]:



import json
import requests
import random as rand
import numpy as np


API_KEY_google = ''
API_KEY_openweather = ''  # Replace with your actual API key

# def gen_wdir():
#     directions = ["N", "NE", "E", "SE", "S", "SW", "W", "NW"]
#     return rand.choice(directions)

# def gen_wind():
#     shape_param = 2.0  # Shape parameter (k)
#     scale_param = 10.0  # Scale parameter (lambda), average wind speed in Sweden
#     wind_speed = scale_param * np.random.weibull(shape_param)
#     return round(wind_speed, 2)

def conditions():
    return gen_wdir(), gen_wind()

def user_skill():
    skills = ["1: Beginner", "2: Intermediate", "3: Advanced"]
    while True:
        try:
            print(f"Choose your skill level:\n{', '.join(skills)}")
            choice = int(input("Enter skill number: "))
            if 1 <= choice <= 3:
                return choice  # Return the integer skill level
            else:
                print("Invalid input. Please enter a number between 1 and 3.")
        except ValueError:
            print("Invalid input. Please enter a valid number.")

# Asks for which sport the user will do
def user_sport():
    sports = ["1: Kite", "2: Foil"]
    while True:
        try:
            print(f"Choose your sport:\n{', '.join(sports)}")
            choice = int(input("Enter sport number: "))
            if 1 <= choice <= 2:
                return sports[choice - 1]
            else:
                print("Invalid input. Please enter 1 or 2.")
        except ValueError:
            print("Invalid input. Please enter a valid number.")


# Geocode city to lat,lon
def get_city_coordinates(city):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={city}&key={API_KEY_google}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            location = data['results'][0]['geometry']['location']
            return [location['lat'], location['lng']]
        else:
            print("No data found for the specified city.")
            return None
    else:
        print(f"Error: Unable to fetch data (HTTP {response.status_code})")
        return None


# Get distance between home and spot. 
def get_driving_distance(origin, destination):
    """
    Function to calculate driving distance and duration between two locations using Google Maps Directions API.
    
    Parameters:
    origin (str): The starting location (e.g., "New York, NY").
    destination (str): The destination location (e.g., "Los Angeles, CA").
    
    Returns:
    tuple: (distance in float, duration as str) if the request is successful, or None if not.
    """
    # Build the URL for the API request
    url = f"https://maps.googleapis.com/maps/api/directions/json?origin={origin}&destination={destination}&mode=driving&key={API_KEY_google}"

    # Make the API request
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()

        # Check if there are any routes in the response
        if data['routes']:
            legs = data['routes'][0]['legs'][0]
            
            # Get the distance in a human-readable format (e.g., "38.7 km" or "500 m")
            distance_str = legs['distance']['text']
            
            # Convert to kilometers (if 'm' found, convert meters to kilometers)
            if 'km' in distance_str:
                distance = float(distance_str.replace(' km', ''))
            elif 'm' in distance_str:
                distance = float(distance_str.replace(' m', '')) / 1000
            else:
                raise ValueError(f"Unrecognized distance format: {distance_str}")

            # Get the duration in a human-readable format (e.g., "4 hours 21 mins")
            duration = legs['duration']['text']
            
            return distance, duration
        else:
            print("No route found between the specified locations.")
            return None
    else:
        print(f"Error: Unable to fetch data (HTTP {response.status_code})")
        return None

# Update the spots.txt with new spot.
def add_spot(filename):
    city = input("Enter City: ")
    coord = get_city_coordinates(city)
    if not coord:
        print(f"Could not fetch coordinates for {city}.")
        return
    
    wind_direction = int(input("Enter wind direction with degrees (360 degrees): "))
    try:
        optimal_wind_speed = int(input("Enter optimal wind speed: "))
        level = int(input("Enter level: "))
    except ValueError:
        print("Invalid input. Wind speed and level must be numbers.")
        return

    new_spot = {
        "spot": city,
        "coord": coord,
        "optimal_wind_direction": wind_direction,
        "optimal_wind_speed": optimal_wind_speed,
        "level": level
    }

    try:
        with open(filename, 'r', encoding='utf-8') as file:
            data = json.load(file)
    except (FileNotFoundError, json.JSONDecodeError):
        data = []

    data.append(new_spot)

    with open(filename, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=2)

    print(f"New spot {city} added successfully.")


# Function to choose between adding spot or finding spot
def welcome_menu():
    print("Welcome to OptiWind! Please choose an option:")
    while True:
        try:
            choice = int(input("1: Add spot\n2: Find spot\nEnter your choice: "))
            if choice in [1, 2]:
                return choice
            else:
                print("Invalid input. Please enter 1 or 2.")
        except ValueError:
            print("Invalid input. Please enter a valid number.")


# Get users home location. Used to calculate driving distance
def user_home():
    try:
        lat, long = map(float, input("Enter the coordinates of your home (lat, long): ").split(","))
        return lat, long
    except ValueError:
        print("Invalid input. Please enter valid coordinates.")
        return user_home


# Function do create a Google maps link with driving instrux. 
def get_google_maps_link(origin, destination):
    """
    Function to generate a Google Maps link for driving directions between two locations.
    
    Parameters:
    origin (str): The starting location (e.g., "New York, NY").
    destination (str): The destination location (e.g., "Los Angeles, CA").
    
    Returns:
    str: A URL that can be used to get directions on Google Maps.
    """
    base_url = "https://www.google.com/maps/dir/?api=1"
    travel_mode = "driving"
    
    # Generate the full URL
    maps_url = f"{base_url}&origin={origin}&destination={destination}&travelmode={travel_mode}"
    
    return maps_url
    
# Display the result in a pretty way.
def display_spots(spots, home_location):
    """
    Displays the list of spots with driving distance, time, and a Google Maps link.

    Parameters:
    spots (list): A list of tuples, each containing spot information, driving distance, and time.
    home_location (str): The home location to generate driving links.
    """
    print("\nHere Are Your Spots:")
    print(f"{'No.':<5} {'City':<20} {'Distance':<15} {'Drive Time':<20} {'Google Maps Link'}")
    print("-" * 100)
    
    for i, (spot_dict, dist, time) in enumerate(spots, start=1):
        city_name = spot_dict['spot']  # Extract city name from the dictionary
        
        # Generate the Google Maps link
        maps_link = get_google_maps_link(home_location, city_name)
        
        print(f"{i:<5} {city_name:<20} {dist:<15} {time:<20} {maps_link}")

# Function to create a list with spot, wind and degr, to use when enumerates which spots works with current weahter directions
def update_weather(data):
    up_weather = []
    for i in data:
        surfspot = i["spot"]
        if get_weather(surfspot) != None:
            wind_speed = get_weather(surfspot)["wind"]["speed"]
            wind_deg = get_weather(surfspot)["wind"]["deg"]
            up_weather.append((surfspot, wind_speed, wind_deg))
        else:
            pass
    return up_weather


# Grabs weahter from openweather API
def get_weather(city):
    # This function gets the current weather data for a specific city using OpenWeather API.
    #Parameters:
    #city (str): The name of the city.
    #Returns:
    #dict: A dictionary with weather data, or None if the city is not found.
    # Build the URL for the API request
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY_openweather}&units=metric"
    # Make the API request
    response = requests.get(url)
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        #print(f"Error: Unable to fetch data for {city} (HTTP {response.status_code})")
        return None

def main():
    while True: 
        choice = welcome_menu()
    
        if choice == 1:
            while choice == 1:
                add_spot('spots.txt')
                more = input("Would you like to add more spots? (Y/N): ").strip().lower()
                choice = 1 if more == "y" else 2
    
        print("\nWelcome to the spot finder. Let's find the best spots for you!")
        
        max_dist = int(input("What is the maximum distance you are prepared to drive (in km)? "))
        home = input("Enter your city: ")  # This is the home location
        level = user_skill()
        sport = user_sport()
        
        print(f"\nYour inputs: hometown = {home}, sport = {sport}, level = {level}")
        
        
        with open('spots.txt', 'r', encoding='utf-8') as file:
            data = json.load(file)  # Load the existing data
    
        current_weather = update_weather(data)
        #print(f"Current weather: Wind Direction = {current_weather[0]}, Wind Speed = {current_weather[1]} m/s")
        
        
        suitable_spots = []
        for i in current_weather:
            for j in data:
                if i[0] == j["spot"]:
                    if j['optimal_wind_speed'] <= i[1] and int(j["level"]) <= 2 and (i[2] -300) <= j['optimal_wind_direction'] <= (i[2] +300):
                        suitable_spots.append(j)
                else:
                    pass
        
        if not suitable_spots:
            print("No suitable spots found, try again later.")
        
        else:
            drivable_spots = []
            for spot in suitable_spots:
                result = get_driving_distance(home, spot["spot"])
                if result is not None:
                    dist, time = result
                    if dist <= max_dist:
                        drivable_spots.append((spot, dist, time))
    
            if drivable_spots:
                sorted_spots = sorted(drivable_spots, key=lambda x: x[1])
                display_spots(sorted_spots, home)  # Pass 'home' as the second argument to 'display_spots'
            else:
                print("No spots within your driving distance.")
            print("Thank you for using OptiWind")
            question = input("Do you want to run again [press X]. Else [press any key]")
            if question.lower() == "x":
                continue
            else:
                break
    print("Thank you for using OptiWind\n See you later")                   
if __name__ == "__main__":
    main()



Welcome to OptiWind! Please choose an option:


1: Add spot
2: Find spot
Enter your choice:  2



Welcome to the spot finder. Let's find the best spots for you!


What is the maximum distance you are prepared to drive (in km)?  600
Enter your city:  skövde


Choose your skill level:
1: Beginner, 2: Intermediate, 3: Advanced


...

Choose your sport:
1: Kite, 2: Foil


Enter sport number:  1



Your inputs: hometown = skövde, sport = 1: Kite, level = 3

Here Are Your Spots:
No.   City                 Distance        Drive Time           Google Maps Link
----------------------------------------------------------------------------------------------------
1     Jönköping            85.4            1 hour 14 mins       https://www.google.com/maps/dir/?api=1&origin=skövde&destination=Jönköping&travelmode=driving
2     Örebro               147.0           1 hour 50 mins       https://www.google.com/maps/dir/?api=1&origin=skövde&destination=Örebro&travelmode=driving
3     Smögen               188.0           2 hours 27 mins      https://www.google.com/maps/dir/?api=1&origin=skövde&destination=Smögen&travelmode=driving
4     Varberg              226.0           2 hours 40 mins      https://www.google.com/maps/dir/?api=1&origin=skövde&destination=Varberg&travelmode=driving
5     Varberg              226.0           2 hours 40 mins      https://www.google.com/maps/dir/?api=1&origin=sk

Do you want to run again [press X]. Else [press any key] k


Thank you for using OptiWind
 See you later
